In [1]:
import pandas as pd
#set root directory to parent from this current directory
import os
os.chdir(os.path.dirname(os.getcwd()))
import requests


In [2]:
with open(r"C:\Users\cruz" + r'\API_openAI.txt', 'r') as f:
    read_api_key = f.readline()

# Test local API 

In [3]:
data = {
    "product_description": "avion",
    "database_name": "usnpsc" 
}

# Test local API

In [4]:
# The URL where your Flask API is running
api_url = "https://carbon-ai.onrender.com/search"

# The data you want to send to the API (example values)
data = {
    "product_description": "spectrophotometre",  # Example product description
    "database_name": "labo1point5",  # Example database name
    "top_n": 3  # Example top_n value
}

# Make a POST request to the API
response = requests.post(api_url, json=data)

# Check if the request was successful
if response.status_code == 200:
    # Get the JSON response data
    result = response.json()
    print("API Response:", result)
else:
    print("Failed to call API:", response.status_code, response.text)


API Response: [{'CO2_emitted': None, 'emission_factor': 0.2, 'emission_factor_unit': 'kg CO2e', 'functional_unit': 'euros spent', 'matched_category': 'SPECTROMETRES UV-VISIBLE ET INFRA ROUGE (HORS LECTEURS DE MICROPLAQUES)', 'matched_code': 'SE12', 'similarity_score': 0.85}, {'CO2_emitted': None, 'emission_factor': 0.2, 'emission_factor_unit': 'kg CO2e', 'functional_unit': 'euros spent', 'matched_category': 'SPECTROPHOTOMETRIE UV-VISIBLE : LECTEURS DE MICROPLAQUES (SANS RADIOACTIVITE)', 'matched_code': 'SE11', 'similarity_score': 0.85}, {'CO2_emitted': None, 'emission_factor': 0.2, 'emission_factor_unit': 'kg CO2e', 'functional_unit': 'euros spent', 'matched_category': 'SPECTROPHOTOMETRIE UV-VISIBLE : AUTRE MATERIEL ET CONSOMMABLES DEDIES', 'matched_code': 'SE13', 'similarity_score': 0.85}]


In [34]:
result

[{'CO2_emitted': 391385.80748096755,
  'emission_factor': 391.38580748096757,
  'emission_factor_unit': 'g CO2e/unite fonctionnelle',
  'functional_unit': 'pkm',
  'matched_category': 'Avion',
  'matched_code': 'Transport de passagers Aérien Avion  Intercontinental, business',
  'similarity_score': 0.84},
 {'CO2_emitted': 189451.91908193487,
  'emission_factor': 189.45191908193488,
  'emission_factor_unit': 'g CO2e/unite fonctionnelle',
  'functional_unit': 'pkm',
  'matched_category': 'Avion',
  'matched_code': 'Transport de passagers Aérien Avion  Intercontinental, economy',
  'similarity_score': 0.84},
 {'CO2_emitted': 237190.5832595212,
  'emission_factor': 237.1905832595212,
  'emission_factor_unit': 'g CO2e/unite fonctionnelle',
  'functional_unit': 'pkm',
  'matched_category': 'Avion',
  'matched_code': 'Transport de passagers Aérien Avion  Intercontinental, moyenne',
  'similarity_score': 0.84}]

# Local API


In [3]:
from CO2_calculator_string_API import *

In [4]:
data = {
    "product_description": "spectrophotometre",  # Example product description
    "api_key": read_api_key,
    "database_name": "labo1point5",  # Example database name
    "top_n": 3 , # Example top_n value
    "GPT_instructions": "In the subject of quantuum physics, give me the most accurate and precise."
}

In [44]:

def llm_evaluation(option, instruction):
    # Construct a prompt asking for a numerical score based on the instruction and the option
    prompt = f"Based on the instruction: '{instruction}', rate the relevance of the following option on a scale from 1 to 10. Higher scores indicate better relevance. Option: {option}"

    # Make an API call to the OpenAI ChatCompletion model
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # Specify the GPT-3.5-turbo model
        messages=[
            {"role": "system", "content": "You are a knowledgeable assistant tasked with rating options"},
            {"role": "user", "content": prompt}
        ],
        temperature=0.1  # A lower temperature might encourage more deterministic outputs
    )

    # Extract the text response from the API and attempt to convert to integer
    try:
        # Extracting the first line in case the model provides additional unwanted text
        text_response = response['choices'][0]['message']['content'].strip().split('\n')[0]
        # Convert the response to an integer
        score = int(text_response)
    except ValueError:
        # In case the conversion fails, log an error or handle it as appropriate
        print(f"Failed to convert response to integer for option: {option}")
        score = 0  # Default score in case of failure

    return score


# Example usage
matches = ['SPECTROMETRES UV-VISIBLE ET INFRA ROUGE (HORS LECTEURS DE MICROPLAQUES)', 'SPECTROPHOTOMETRIE UV-VISIBLE : LECTEURS DE MICROPLAQUES (SANS RADIOACTIVITE)', 'SPECTROPHOTOMETRIE UV-VISIBLE : AUTRE MATERIEL ET CONSOMMABLES DEDIES']  # This should be your actual matches list
best_match = llm_evaluation(matches, "In the subject of quantuum physics, give me the most accurate and precise.")
print("Best Match:", best_match)


In [3]:
from fredapi import Fred
import pandas as pd
import numpy as np

# Replace 'YOUR_API_KEY' with your actual FRED API key
fred = Fred(api_key='7e03169672328a07a6c531d3698d5d54')

# Function to fetch CPI data and calculate cumulative inflation rate
def get_cpi_data(start_date, end_date):
    cpi = fred.get_series('CPIAUCSL', observation_start=start_date, observation_end=end_date)
    return cpi

def calculate_cumulative_inflation(cpi_data):
    # Calculate monthly percentage changes
    cpi_monthly_pct_change = cpi_data.pct_change().dropna() * 100
    
    # Calculate cumulative inflation
    cumulative_inflation = (np.prod(1 + cpi_monthly_pct_change / 100) - 1) * 100
    return cumulative_inflation

# Example usage to calculate inflation rate from 2019-01-01 to 2023-05-05
start_date = '2019-01-01'
end_date = '2023-05-05'
cpi_data = get_cpi_data(start_date, end_date)
cumulative_inflation_rate = calculate_cumulative_inflation(cpi_data)
print(f'Cumulative inflation rate from {start_date} to {end_date} is {cumulative_inflation_rate:.2f}%')


Cumulative inflation rate from 2019-01-01 to 2023-05-05 is 20.12%


In [6]:
from fredapi import Fred
import pandas as pd

# Replace 'YOUR_API_KEY' with your actual FRED API key
fred = Fred(api_key='7e03169672328a07a6c531d3698d5d54')

# Function to fetch HICP data for the Eurozone
def get_hicp_data(start_date, end_date):
    hicp = fred.get_series('CP0000EZ19M086NEST', observation_start=start_date, observation_end=end_date)
    return hicp

# Function to calculate the inflation factor
def calculate_inflation_factor(current_date, historical_date):
    # Fetch HICP data for the range including both dates
    start_date = min(current_date, historical_date)
    end_date = max(current_date, historical_date)
    
    hicp_data = get_hicp_data(start_date, end_date)
    
    # Get the HICP values for the specific dates
    hicp_current = hicp_data[current_date]
    hicp_historical = hicp_data[historical_date]
    
    # Calculate the inflation factor
    inflation_factor = hicp_current / hicp_historical
    return inflation_factor

# Function to adjust price for inflation
def adjust_price_for_inflation(price, current_date, historical_date):
    inflation_factor = calculate_inflation_factor(current_date, historical_date)
    adjusted_price = price / inflation_factor
    return adjusted_price

# Example usage
current_date = '2023-01-01'
historical_date = '2019-01-01'
price = 100  # Example price in EUR

inflation_factor = calculate_inflation_factor(current_date, historical_date)
adjusted_price = adjust_price_for_inflation(price, current_date, historical_date)
print(f'Inflation factor from {historical_date} to {current_date} is {inflation_factor:.4f}')
print(f'Price without inflation: {adjusted_price:.2f} EUR')


Inflation factor from 2019-01-01 to 2023-01-01 is 1.1680
Price without inflation: 85.62 EUR
